In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install xgboost
!pip install lightgbm

In [ ]:
train_data = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv" ) # reading the train data to a data frame 
test_data = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv' ) # reading the test data into a data frame 
sample_submission = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv') # reading the test data into a data frame
print(" data imported keep going....")

In [ ]:
train_data.shape #7111,12
train_data.info()

In [ ]:
train_data.head()

In [ ]:
# daypart function
def daypart(hour):
    if hour in [2,3,4,5]:
        return "dawn"
    elif hour in [6,7,8,9]:
        return "morning"
    elif hour in [10,11,12,13]:
        return "noon"
    elif hour in [14,15,16,17]:
        return "afternoon"
    elif hour in [18,19,20,21]:
        return "evening"
    else: return "midnight"

In [ ]:


# from sklearn.preprocessing import StandardScaler
# def datetime_temprature_fetures(data_frame,col_name):
#     scaler = StandardScaler()
#     cols_to_standrdise = ['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']
#     scaler.fit(data_frame[cols_to_standrdise])
#     if data_frame.dtypes[col_name] != 'datetime64[ns]':
#         # convert date_time column to datetime type
#         data_frame[col_name] = pd.to_datetime(data_frame[col_name],errors='coerce')

#     else:
#         print('It\'salready datetime')

#     data_frame['month'] = data_frame[col_name].dt.month # extracting month
#     data_frame['year'] = data_frame[col_name].dt.year # extracting year
#     data_frame['day_of_month'] = data_frame[col_name].dt.day # extracting day
#     # first: extract the day name literal
#     data_frame['day_name'] = data_frame[col_name].dt.day_name()
#     data_frame['hour'] = data_frame[col_name].dt.hour
#     data_frame['hour_categorical'] = data_frame.apply(lambda x : daypart(x['hour']),axis=1)
#     data_frame['is_weekend'] = data_frame.apply(lambda x : 1 if x['day_name'] in ['Saturday','Sunday'] else 0,axis=1)
#     standard_cols = pd.DataFrame(scaler.transform(data_frame[cols_to_standrdise]))
#     data_frame = pd.concat([data_frame,standard_cols],axis=1)
    
   
#     # check if columns exist or not if yes drop them
#     if all(item  in list(train_data.columns) for item in cols_to_standrdise):
#         print('not dropping')
#         data_frame.drop(['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5'],inplace=True,axis=1)
    
#     data_frame.rename(columns= {0:'sensor_1',
#                       1:'sensor_2', 2:'sensor_3',3:'sensor_4',4:'sensor_5'},inplace=True)
    
#     return data_frame


In [ ]:
# train_data = datetime_temprature_fetures(train_data,'date_time')
# train_data.head()

In [ ]:
# test_data = datetime_temprature_fetures(test_data,'date_time')
# test_data.head()

In [ ]:
for c in train_data.columns:
    col_type = train_data[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        train_data[c] = train_data[c].astype('category')

for c in test_data.columns:
    col_type = test_data[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        test_data[c] = test_data[c].astype('category')


### Dataset ready

1. Now after analysing Training and Testing Data I found Year is different in both so droping  **Year** column
2. Creating **Lag features** i.e value of sensor 1 hour ago , 1 day ago found 

   [https://www.kaggle.com/c/tabular-playground-series-jul-2021/discussion/250074#1374044](here)
        

In [ ]:
# train_data.drop(['year','date_time'],axis=1,inplace=True)

# test_data.drop(['year','date_time'],axis=1,inplace=True)

In [ ]:
print(train_data.shape)
print(test_data.shape)

targets = ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']

y = train_data.loc[:,targets]

train_data.drop(labels = targets,inplace=True,axis=1)

In [ ]:

y1 = np.array(y.loc[:, list(y.columns)[0] ] )# first column for prediction

y2 = np.array(y.loc[:, list(y.columns)[1] ]) # second column for prediction

y3 = np.array(y.loc[:, list(y.columns)[2] ]) # third column for prediction

In [ ]:
cat_cols = train_data.select_dtypes('category').columns.tolist()
cat_cols

In [ ]:
from sklearn.model_selection import StratifiedKFold ,KFold

In [ ]:

skf = KFold(n_splits=10, shuffle=True,random_state=42)

def trainer(model, X,y,TEST_SET):
    i=1
    pred_val_final = np.zeros((1,7111))
    pred_test_final = np.zeros((1,2247))
    new_train_label_final = np.zeros((1,7111))
    
    fit_params={ 'feature_name':'auto',
            'categorical_feature': 'auto' # that's actually the default
           }
    
    for train_index, test_index in skf.split(X, y):
        
        print(f"For Split Number {i}")
        X_train  = X.loc[train_index,:]
        X_test = X.loc[test_index,:]
        y_train = y[train_index]
        y_test = y[test_index]

#         print(f"Training shape {X_train.shape} and {y_train.shape}") # Training shape (6399, 14) and (6399,)
#         print(f"Testing shape {X_test.shape} and {y_test.shape}")  # Testing shape (712, 14) and (712,)

  
        model.fit(X_train,y_train) # if lightgbm model is LGBM it can handle catgorical features on it's own given
                                    # we are providing Pandas dataframe to it not numpy array

        #storing predictions for train and test
        pred_val= model.predict(X_test)

        i+=1
        pred_val = pred_val.reshape(-1,1).T


        # new y for next training data
        new_train_label = y_test.reshape(-1,1).T


        new_train_label_final = np.hstack((new_train_label_final,new_train_label)) #new y
        pred_val_final = np.hstack((pred_val_final,pred_val)) # new x


#         print(pred_val_final.shape,pred_test_final.shape,i,new_train_label_final.shape)

   
    # new test data which we get after prediction after 10 cross folds we will use 
    # last model to predict our TEST DATA
    model.fit(X,y)
    pred_test=model.predict(TEST_SET)
    pred_test = pred_test.reshape(-1,1).T
    print(pred_test.shape,"after predition")
    pred_test_final = np.hstack((pred_test_final,pred_test)) 
    print(pred_test_final.shape,"after stacking")
    return pred_val_final,pred_test_final,new_train_label_final


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRFRegressor
from lightgbm import LGBMRegressor,LGBMClassifier
model=LGBMRegressor(num_leaves= 15, max_depth=-1, 
                         random_state=314, 
                         silent=True, 
                         metric='None', 
                         n_jobs=4, 
                         n_estimators=1000,
                         colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.1)


new_train_x_model_1, new_test_x_model_1, new_train_y_model_1 = trainer(model , train_data,y1,test_data)
new_train_x_model_2, new_test_x_model_2, new_train_y_model_2 = trainer(model , train_data,y2,test_data)
new_train_x_model_3, new_test_x_model_3, new_train_y_model_3 = trainer(model , train_data,y3,test_data)

In [ ]:

print("*"*50,"\n")
print(new_train_x_model_1.shape,"---- validation  x shape for model 1")
print(new_train_y_model_1.shape,"---- validation  y shape for model 1")
print(new_test_x_model_1.shape,"---- test shape for model 1")


print("*"*50,"\n")
print(new_train_x_model_2.shape,"---- validation  x shape for model 2")
print(new_train_y_model_2.shape,"---- validation  y shape for model 2")
print(new_test_x_model_2.shape,"---- test shape for model 2")


print("*"*50,"\n")
print(new_train_x_model_3.shape,"---- validation  x shape for model 3")
print(new_train_y_model_3.shape,"---- validation  y shape for model 3")
print(new_test_x_model_3.shape,"---- test shape for model 3")

In [ ]:
print("For model 1")
new_train_x_model_1 = new_train_x_model_1[:,7111:] # removing zeroes column
new_train_y_model_1 = new_train_y_model_1[:,7111:] # new y column

new_test_x_model_1 = new_test_x_model_1[:,2247:]  # removing zeroes column

print("For model 2")
new_train_x_model_2 = new_train_x_model_2[:,7111:] # removing zeroes column
new_train_y_model_2 = new_train_y_model_2[:,7111:] # new y column

new_test_x_model_2= new_test_x_model_2[:,2247:]  # removing zeroes column

print("For model 3")
new_train_x_model_3 = new_train_x_model_3[:,7111:] # removing zeroes column
new_train_y_model_3 = new_train_y_model_3[:,7111:] # new y column

new_test_x_model_3= new_test_x_model_3[:,2247:]  # removing zeroes column

In [ ]:
print("For model 1")
new_train_x_model_1 = new_train_x_model_1.T
new_train_y_model_1 = new_train_y_model_1.T
new_test_x_model_1 = new_test_x_model_1.T
print(new_train_x_model_1.shape ," First ROW NEW TRAIN X")
print(new_train_y_model_1.shape,"FIRST NEW TRAIN Y ")
print(new_test_x_model_1.shape)

print("For model 2")
new_train_x_model_2 = new_train_x_model_2.T
new_train_y_model_2 = new_train_y_model_2.T
new_test_x_model_2 = new_test_x_model_2.T
print(new_train_x_model_2.shape ," First ROW NEW TRAIN X")
print(new_train_y_model_2.shape,"FIRST NEW TRAIN Y ")
print(new_test_x_model_2.shape)

print("For model 1")
new_train_x_model_3 = new_train_x_model_3.T
new_train_y_model_3 = new_train_y_model_3.T
new_test_x_model_3 = new_test_x_model_3.T
print(new_train_x_model_3.shape ," First ROW NEW TRAIN X")
print(new_train_y_model_3.shape,"FIRST NEW TRAIN Y ")
print(new_test_x_model_3.shape)

 ### XGBOOST

In [ ]:
train_data_xgb = train_data.copy()
test_data_xgb = test_data.copy()

train_data_xgb.drop(['date_time'],inplace=True,axis=1)
test_data_xgb.drop(['date_time'],inplace=True,axis=1)
# # OHE for XGBOOST for training data
# train_data_xgb =  pd.get_dummies(train_data_xgb, columns = ['hour_categorical','day_name'])

# test_data_xgb =  pd.get_dummies(test_data_xgb, columns = ['hour_categorical','day_name'])


In [ ]:
#Model 2- XGBREgressor
Xgreg = XGBRFRegressor(random_state= 101)
new_train_x_xg_1 ,new_test_x_xg_1, new_train_y_xg_1 = trainer(Xgreg,train_data_xgb ,y1 ,test_data_xgb)

new_train_x_xg_2 ,new_test_x_xg_2, new_train_y_xg_2 = trainer(Xgreg,train_data_xgb ,y2 ,test_data_xgb)

new_train_x_xg_3 ,new_test_x_xg_3, new_train_y_xg_3 = trainer(Xgreg,train_data_xgb ,y3 ,test_data_xgb)

In [ ]:

print("*"*50,"\n")
print(new_train_x_xg_1.shape,"---- validation  x shape for model 1")
print(new_train_y_xg_1.shape,"---- validation  y shape for model 1")
print(new_test_x_xg_1.shape,"---- test shape for model 1")


print("*"*50,"\n")
print(new_train_x_xg_2.shape,"---- validation  x shape for model 2")
print(new_train_y_xg_2.shape,"---- validation  y shape for model 2")
print(new_test_x_xg_2.shape,"---- test shape for model 2")


print("*"*50,"\n")
print(new_train_x_xg_3.shape,"---- validation  x shape for model 3")
print(new_train_y_xg_3.shape,"---- validation  y shape for model 3")
print(new_test_x_xg_3.shape,"---- test shape for model 3")

In [ ]:



print("For model 1")
new_train_x_xg_1 = new_train_x_xg_1[:,7111:] # removing zeroes column
new_train_y_xg_1 = new_train_y_xg_1[:,7111:] # new y column

new_test_x_xg_1 = new_test_x_xg_1[:,2247:]  # removing zeroes column

print("For model 2")
new_train_x_xg_2 = new_train_x_xg_2[:,7111:] # removing zeroes column
new_train_y_xg_2 = new_train_y_xg_2[:,7111:] # new y column

new_test_x_xg_2= new_test_x_xg_2[:,2247:]  # removing zeroes column

print("For model 3")
new_train_x_xg_3 = new_train_x_xg_3[:,7111:] # removing zeroes column
new_train_y_xg_3 = new_train_y_xg_3[:,7111:] # new y column

new_test_x_xg_3= new_test_x_xg_3[:,2247:]  # removing zeroes column

In [ ]:
print("For model 1")
new_train_x_xg_1 = new_train_x_xg_1.T
new_train_y_xg_1 = new_train_y_xg_1.T
new_test_x_xg_1 = new_test_x_xg_1.T
print(new_train_x_xg_1.shape ," First ROW NEW TRAIN X")
print(new_train_y_xg_1.shape,"FIRST NEW TRAIN Y ")
print(new_test_x_xg_1.shape)

print("For model 2")
new_train_x_xg_2 = new_train_x_xg_2.T
new_train_y_xg_2 = new_train_y_xg_2.T
new_test_x_xg_2 = new_test_x_xg_2.T
print(new_train_x_xg_2.shape ," First ROW NEW TRAIN X")
print(new_train_y_xg_2.shape,"FIRST NEW TRAIN Y ")
print(new_test_x_xg_2.shape)

print("For model 1")
new_train_x_xg_3 = new_train_x_xg_3.T
new_train_y_xg_3 = new_train_y_xg_3.T
new_test_x_xg_3 = new_test_x_xg_3.T
print(new_train_x_xg_3.shape ," First ROW NEW TRAIN X")
print(new_train_y_xg_3.shape,"FIRST NEW TRAIN Y ")
print(new_test_x_xg_3.shape)

### NEW TRAINING DATA AND NEW TESTING DATA

In [ ]:
train_new_1 = np.hstack((new_train_x_model_1,new_train_x_xg_1))
test_new_1 = np.hstack((new_test_x_model_1,new_test_x_xg_1))

train_new_y_1 =  np.hstack((new_train_y_model_1,new_train_y_xg_1))

train_new_2 = np.hstack((new_train_x_model_2,new_train_x_xg_2))
test_new_2 = np.hstack((new_test_x_model_2,new_test_x_xg_2))

train_new_y_2 =  np.hstack((new_train_y_model_2,new_train_y_xg_2))

train_new_3 = np.hstack((new_train_x_model_3,new_train_x_xg_3))
test_new_3 = np.hstack((new_test_x_model_3,new_test_x_xg_3))

train_new_y_3 =  np.hstack((new_train_y_model_3,new_train_y_xg_3))

In [ ]:
print("model1")
print(train_new_1.shape)
print(test_new_1.shape)
print(train_new_y_1.shape)

print("model 2")
print(train_new_2.shape)
print(test_new_2.shape)
print(train_new_y_2.shape)
print("model 3")
print(train_new_3.shape)
print(test_new_3.shape)
print(train_new_y_3.shape)

In [ ]:

final_model_1 = RandomForestRegressor(random_state= 101)
final_model_1.fit(train_new_1,train_new_y_1[:,1])


final_model_2 = RandomForestRegressor(random_state= 101)
final_model_2.fit(train_new_2,train_new_y_2[:,1])


final_model_3 = RandomForestRegressor(random_state= 101)
final_model_3.fit(train_new_3,train_new_y_3[:,1])

In [ ]:
y_pred_1 = final_model_1.predict(test_new_1)

y_pred_2 = final_model_2.predict(test_new_2)

y_pred_3 = final_model_3.predict(test_new_3)

In [ ]:
test_sub = np.vstack([y_pred_1,y_pred_2,y_pred_3]).T

In [ ]:
sample_submission[sample_submission.columns[1:]] = test_sub
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv',index=False)